# pre_process

In [1]:
import re,jieba,random,time
import numpy as np
import torch
from sklearn.model_selection import train_test_split

In [2]:
word2id,id2word,wordnum = None,None,None

In [3]:
# 标点符号处理
def filter_sent(sent):
    return sent.replace('\n','').replace(' ','').replace('，',',').replace('。','.').replace('；',';').replace('：',':').replace('？','?').replace('！','!').replace('“','"').replace('”','"').replace("‘","'").replace("’","'").replace('（','(').replace('）',')')

In [13]:
filepath = '../data/qingyun.tsv'
with open(filepath,'r',encoding='utf-8') as f:
    txt = f.readlines()

In [16]:
data = [filter_sent(i) for i in txt] #一层列表

In [20]:
data = [i.split('\t') for i in txt] #二层列表

In [22]:
data = [[jieba.lcut(i[0]),jieba.lcut(i[1])] for i in data] #三层列表

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.323 seconds.
Prefix dict has been built succesfully.


In [23]:
chatdataword = data

In [28]:
id2word = list((set([w for qa in data for sent in qa for w in sent])))

In [30]:
id2word.sort()

In [32]:
id2word = ['<EOS>','<SOS>'] + id2word + ['<UNK>']

In [34]:
all_words = id2word

In [38]:
id2word = {i[0]:i[1] for i in enumerate(all_words)}
word2id = {i[1]:i[0] for i in enumerate(all_words)}
wordNum = len(all_words) - 2

In [44]:
chatDataId = [[[word2id[w] for w in qa[0]],[word2id[w] for w in qa[1]]] for qa in chatdataword]

In [47]:
QLens,ALens = [len(qa[0])+1 for qa in data],[len(qa[1])+1 for qa in data]
QMaxLen,AMaxLen = max(QLens),max(ALens)
QLens = np.array(QLens,dtype='int32')
ALens = np.array(ALens,dtype='int32')

In [51]:
maxSentLen = 177

In [52]:
QChatDataId,AChatDataId = [qa[0] for qa in chatDataId],[qa[1] for qa in chatDataId]

In [53]:
totalSampleNum = len(data)

105914

In [55]:
[np.log(totalSampleNum/(sum([(i in qa[0]) for qa in chatDataId])+1)) for i in range(wordNum)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [61]:
chatDataId[0][0]

[10477, 13780, 12805, 3]

In [58]:
x

[0, 4, 9, 7, 6, 5, 8, 3]

# model

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from matplotlib import pyplot as plt
from matplotlib import ticker
from nltk.translate.bleu_score import sentence_bleu
import time,random,os,jieba,logging
import numpy as np
import pandas as pd
jieba.setLogLevel(logging.INFO)

In [2]:
sosToken, eosToken = 1, 0

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, featureSize, hiddenSize, embedding, numLayers=1,dropout=.01,
                bidirectinal=True):
        super(EncoderRNN, self).__init__()
        self.embedding = embedding
        self.gru = nn.GRU(featureSize, hiddenSize, num_layers=numLayers,
                          dropout=(0 if numLayers == 1 else dropout),bidirectional=bidirecional,
                          batch_first=True)
        self.featureSize = featureSize
        self.hiddenSize = hiddenSize
        self.numLayers = numLayers
        self.bidirectional = bidirectional
        
    def forward(self,input,lengths,hidden)

        